## Imports

In [1]:
from ipynb.fs.full.Jacobian import *
from ipynb.fs.full.SubFunctions import *
from ipynb.fs.full.GlobalVars import *

## Solver for one differencial equation

In [2]:
def Solver_one_equation(expr, u0, use_trapezoidal, symbol):
    t0 = 0
    un_1 = u0
    un = u0
    un_save = un
    
    velocity_n = 0
    #Need to keep in memory the previous derivative value.
    if(use_trapezoidal):
        velocity_n = expr.subs(symbol, u0)
    

    function = [u0]
    time = [t0]

    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        f_un_plus1 = 1
       
        #Newton's method
        while(nb < Nnewton):
            nb += 1
            
            f = create_f(symbol, un_1, expr, use_trapezoidal, velocity_n)
            f_un = f.subs(symbol, un)
            f_un_prime = Jacobian_one(f).subs(symbol, un)
            
            un_plus1 = float(un - (f_un / f_un_prime))
            f_un_plus1 = float(f.subs(symbol, un_plus1))
            
            un = un_plus1
            if(f_un_plus1 < Epsilon):
                break
            
            
        #Need to keep in memory the residual of the previous iteration.
        if(use_trapezoidal):
            velocity_n = (2/delta) * (un_plus1 - un_save) - velocity_n
        
        un_1 = un
        un_save = un
        time.append(t0)
        function.append(un_plus1)
        
    return result_solver(time, function)

## Solver for 2 differentials equations

In [3]:
def Solver_two_equations(u0, v0, f1, f2, use_trapezoidal):
    t0 = 0
    
    xk_1 = np.array([u0, v0])
    xk = xk_1
    xk_save = xk
    
    velocity_n = [0, 0]
    #Need to keep in memory the previous derivative value.
    if(use_trapezoidal):
        velocity_n = [f1.subs(v, v0), f2.subs(u, u0)]
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Functions = [u0, u1, u2, ...., un]
    fu = [u0]
    fv = [v0]
    time = [t0]
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        f_xk_plus1 = 1
        #Newton's method
        while(nb < Nnewton):
            nb+=1
            
            vec_f = create_2_f(xk_1, f1, f2, use_trapezoidal, velocity_n)
            F_xk = create_fxk(xk, vec_f)
            F_xk_prime = f_prime_2_eqs(xk, vec_f[0], vec_f[1])
            xk_plus1 = xk - np.dot(LA.inv(F_xk_prime), F_xk)
            
            xk = xk_plus1
            
            #Calcul of the approximation.
            f_xk_plus1 =[0, 0]
            f_xk_plus1[0] = float(vec_f[0].evalf(subs={u: xk_plus1[0], v: xk_plus1[1]}))
            f_xk_plus1[1] = float(vec_f[1].evalf(subs={u: xk_plus1[0], v: xk_plus1[1]})) 
            norm_f_xk_plus1 = LA.norm(f_xk_plus1)
            
            if(norm_f_xk_plus1 < Epsilon):
                break
            
        #Need to keep in memory the residual of the previous iteration.
        if(use_trapezoidal):
            velocity_n[0] =  (2/delta) * (xk_plus1[0] - xk_save[0]) - velocity_n[0]
            velocity_n[1] = (2/delta) * (xk_plus1[1] - xk_save[1]) - velocity_n[1]
            
        
        #Update our 3 lists and indexes.
        time.append(t0)
        fu.append(xk_plus1[0])
        fv.append(xk_plus1[1])
        xk_1 = xk
        xk_save = xk
      
    return result_solver(time, fu, fv)

## Solver for 4 equations

In [4]:
def Solver_4_equations(u0, v0, f1, f2, use_trapezoidal):
    t0 = 0
    
    uk = u0
    vk = v0
    xk_1 = np.array([u0, v0])
    xk = xk_1
    xk_save = xk
   
    velocity_n = [0, 0]
    #Need to keep in memory the previous derivative value.
    if(use_trapezoidal):
        velocity_n = [f1.subs(v, v0), f2.subs(u, u0)]
    
    #Each list is an axis.
    #Time = [t0, t1, t2, ...., tn]
    #Functions = [u0, u1, u2, ...., un]
    fu = [u0]
    fv = [v0]
    time = [t0]
    
    #Suit of x and y variables
    y0 = u0*v0
    x0 = u0 + v0 - y0
    x = [x0]
    y = [y0]
    
    #Loop for the time
    while(t0 < interval):
        t0 += delta
        nb = 0
        f_xk = 1
        
        #Newton's method
        while(nb < Nnewton):
            nb+=1
            
            vec_f = create_2_f(xk_1, f1, f2, use_trapezoidal, velocity_n)
            F_xk = create_fxk(xk, vec_f)
            F_xk_prime = f_prime_2_eqs(xk, vec_f[0], vec_f[1])
        
            #Formula : xk = xk_1 - tr(F'(xk_1)) * F(xk_1)
            xk_plus1 = xk - np.dot(LA.inv(F_xk_prime), F_xk)
            
            #With the uk+1 and vk+1 calculated, we can calcul the x and y value.
            next_y = xk_plus1[0]*xk_plus1[1]
            next_x = xk_plus1[0] + xk_plus1[1] - next_y
            
            #Calcul of f(uk, vk, xk, yk) for the approximation
            f_xk_plus1 = [0, 0, 0, 0]
            f_xk_plus1[0] = float(vec_f[0].evalf(subs={u: xk_plus1[0], v: xk_plus1[1]}))
            f_xk_plus1[1] = float(vec_f[1].evalf(subs={u: xk_plus1[0], v: xk_plus1[1]}))
            f_xk_plus1[2] = float(next_x)
            f_xk_plus1[3] = float(next_y)
            
            xk = xk_plus1
            
            #Approximation.
            norm_f = LA.norm(f_xk_plus1)
            if(norm_f < Epsilon):
                break
            
        if(use_trapezoidal):
            velocity_n[0] =  (2/delta) * (xk_plus1[0] - xk_save[0]) - velocity_n[0]
            velocity_n[1] = (2/delta) * (xk_plus1[1] - xk_save[1]) - velocity_n[1]
        
        
        #Update our 2 functions + time
        time.append(t0)
        fu.append(xk_plus1[0])
        fv.append(xk_plus1[1])
        
        #Update x and y variables
        x.append(next_x)
        y.append(next_y)
            
        #Update indexes
        xk_1 = xk
        xk_save =xk
               
    return result_solver(time, fu, fv, x, y)